# Implementacija algoritma $AQC(p)$

Ovde ćemo prikazati kako se implementira prvi algoritam koji su predložili Dōng i Lín. Nasumično ćemo generisati retku matricu veličine $16 \times 16$, a onda i vektor $b$ dužine $16$, nakon čega ćemo prikazati kako se rešava sistem jednačina $A x = b$.

In [1]:
import scipy.sparse as sparse
import numpy as np

rows = 16
cols = 16
density = 0.1  # 10% non-zero elements

R = sparse.random(rows, cols, density=density, format='csr', dtype=np.float64)
A = R.T @ R + sparse.diags(np.ones(cols) * 10, format='csr')
A = A.toarray()

In [2]:
from math import sqrt

b = np.random.rand(cols)
c = sqrt(sum(b*b))
b = 1/c * b

Matrica $A$ najverovatnije nije ermitska pozitivno definitna, pa koristimo recept koji su dali autori da se matrica $A$ najpre pretvori u ermitsku.

In [3]:
X = np.array([[0.0, 1.0], [1.0, 0.0]])
Y = np.array([[0.0, -1.0j], [1.0j, 0.0]])
Z = np.array([[1.0, 0.0], [0.0, -1.0]])

Op, Om = 1/2.0 * (X + 1.0j * Y), 1/2.0 * (X - 1.0j * Y)

In [4]:
A1 = np.kron(Op, A) + np.kron(Om, A.conj().T)
b1 = np.kron(np.array([0, 1]), b)
N = 2*cols

Sada generišemo Hamiltonijane $H_0$ i $H_1$. Koristićemo proceduru za slučaj kada $A$ nije pozitivno definitna.

In [5]:
plus = 1/sqrt(2.0) * np.array([1, 1])
Plus = plus.reshape(-1, 1) @ plus.reshape(1, 2)
minus = 1/sqrt(2.0) * np.array([1, -1])
B1 = b1.reshape(-1, 1) @ b1.reshape(1, N)

Qb = np.eye(2*N) - np.kron(Plus, B1)
H0 = np.kron(Op, np.kron(Z, np.eye(N)) @ Qb) + np.kron(Om, Qb @ np.kron(Z, np.eye(N)))
H1 = np.kron(Op, np.kron(X, A1) @ Qb) + np.kron(Om, Qb @ np.kron(X, A1))

Proverimo da je sve dobro tako što ćemo ubaciti sopstvene vrednosti uz $0$ od $H_0$.

In [6]:
#print(H0 @ np.kron(np.kron(np.array([0, 1]), plus), b1).reshape(-1, 1))
#print(H0 @ np.kron(np.kron(np.array([1, 0]), minus), b1).reshape(-1, 1))

Sve je tu negde na oko $10^{-16}$. To je verovatno zbog preciznosti računanja sa **float** promenljivama na računaru. Deluje da je ovo sve u redu.

Sada je potrebno definisati funkciju za protok vremena. Ona zavisi od kondicionog broja matrice $A1$. Pravićemo se da imamo efikasan algoritam za ovo i pustićemo **numpy** da izračuna ovo za nas.

In [7]:
import numpy.linalg as la

kappa = la.cond(A1)
kappa

1.206804173308867

Sada imamo kondicioni broj i možemo definisati funkciju vremena.

In [8]:
def time_function(kappa, s, p):
    return kappa/(kappa - 1) * (1 - (1 + s*(kappa**(p - 1) - 1))**(1/(1 - p)))

Za parametar $p$ ćemo uzeti $0.5$. Sada biramo $T = O(\kappa / \epsilon)$. Neka to bude $T = \kappa / \epsilon$, a za $\epsilon = 1/10$.

In [9]:
epsilon = 1.0/10.0
T = kappa / epsilon

T

12.06804173308867

Ostaje samo da se izabere parametar $M$ koji određuje koliko ćemo imati operatora vremenske evolucije. On se bira tako da bude $M = O(\text{polylog}(N) T^2 / \epsilon)$. Postavićemo ga na $20$, jer nemamo dovoljno jaku mašinu da simulira kvantni računar sa toliko mnogo kvantnih kola. Naime, prava vrednost bi ovde bila preko $1000$.

In [10]:
M = 20

Broj kubita je $\log_2$ od veličine matrice $H_0$. U ovom slučaju je to $7$. Sada pravimo kvantno kolo koje će opisati vremensku evoluciju. Na to kvantno kolo najpre treba inicijalizovati $\ket{0} \ket{-} \ket{b_1}$.

In [11]:
from qiskit.circuit import QuantumCircuit
from math import log2
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.circuit.library import StatePreparation
from qiskit.quantum_info import SparsePauliOp

# predstavljamo H0 i H1 u SparsePauliOp

h0, h1 = SparsePauliOp.from_operator(H0), SparsePauliOp.from_operator(H1)
n = int(log2(H0.shape[0]))

time_evolution = QuantumCircuit(n)

initial_state = np.kron(np.array([1, 0]), np.kron(minus, b1))
stateprep = StatePreparation(initial_state)
time_evolution.append(stateprep, range(n))

h = 1.0/M

for i in reversed(range(1, M + 1)):
    si = i*h
    time_evolution.append(PauliEvolutionGate(h1, time = T * time_function(kappa, si, 0.5) * h), range(n))
    time_evolution.append(PauliEvolutionGate(h0, time = T * (1 - time_function(kappa, si, 0.5)) * h), range(n))